
This is a book recommendation algorithm using K-Nearest Neighbors.

It uses the Book-Crossings dataset. This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users.

After importing and cleaning the data, use NearestNeighbors from sklearn.neighbors to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named get_recommends that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")

should return:

In [ ]:
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]

['The Queen of the Damned (Vampire Chronicles (Paperback))',
 [['Catch 22', 0.793983519077301],
  ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479],
  ['Interview with the Vampire', 0.7345068454742432],
  ['The Tale of the Body Thief (Vampire Chronicles (Paperback))',
   0.5376338362693787],
  ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]]]

Notice that the data returned from get_recommends() is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip -O book-crossings.zip

!unzip -o book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-01-08 19:19:56--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   117MB/s    in 0.2s    

2023-01-08 19:19:57 (117 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# Clean up the insignificant data
user_count = df_ratings['user'].value_counts()
rating_count = df_ratings['isbn'].value_counts()

df = df_ratings
df = df[df['user'].isin(user_count[user_count >= 200].index)]
df = df[df['isbn'].isin(rating_count[rating_count >= 100].index)]

In [ ]:
# Merge the 2 dataframe together
df = pd.merge(right=df, left=df_books, on='isbn')
df.head

<bound method NDFrame.head of              isbn                                              title  \
0      0440234743                                      The Testament   
1      0440234743                                      The Testament   
2      0440234743                                      The Testament   
3      0440234743                                      The Testament   
4      0440234743                                      The Testament   
...           ...                                                ...   
49512  0515135739  Eleventh Hour: An FBI Thriller (FBI Thriller (...   
49513  0515135739  Eleventh Hour: An FBI Thriller (FBI Thriller (...   
49514  0515135739  Eleventh Hour: An FBI Thriller (FBI Thriller (...   
49515  0515135739  Eleventh Hour: An FBI Thriller (FBI Thriller (...   
49516  0515135739  Eleventh Hour: An FBI Thriller (FBI Thriller (...   

                  author    user  rating  
0           John Grisham  277478     0.0  
1           John Gr

In [ ]:
# Remove duplicates and NA
df = df.drop_duplicates(subset=['user', 'title'], keep='last')

In [ ]:
# Create pivot table and convert to scipy CSR matrix
pivot_table = df.pivot(index='title', columns='user', values='rating').fillna(0)
matrix = csr_matrix(pivot_table.values)

In [ ]:
# Init KNN model
neigh = NearestNeighbors(n_neighbors=6, metric='cosine')
neigh.fit(matrix)

NearestNeighbors(metric='cosine', n_neighbors=6)

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  book_arr = pivot_table[pivot_table.index == book]

  book_arr = book_arr.to_numpy().reshape((1, -1))
  distances, suggestions = neigh.kneighbors(book_arr)

  distances = distances[0][::-1]
  suggestions = suggestions[0][::-1]

  recommended = []
  for i in range(5):
    recommended.append([pivot_table.index[suggestions[i]], distances[i]])
  recommended_books = [
      book,
      recommended
  ]
  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
